# Building a Neural Network from Scratch

## Requirements 

- A working fully-connected deep neural network from scratch using only numpy.
- Includes dense layers, activations, optimizers, loss functions and sigmoid or softmax in case of classification. 
- Runtime and results on a public dataset.
- Documented code that includes brief summary, technical details, and results. 

## Extensions 

A comparison of the model’s runtime and performance with/without each component:
- More than 1 optimizer - SGD, Momentum, RMSProp, Adam etc.
- Regularization - L2/weight decay, dropout, possibly augmentations if image data etc.
- Results on more than 1 dataset.


### Load the dataset

In [97]:
import numpy as np

from sklearn.datasets import fetch_openml
#load MNIST dataset
mnist = fetch_openml('mnist_784')
X = mnist.data
y = mnist.target

In [96]:
X = X /255
y = np.where(y=='0', 0, 1)

In [101]:
m = 60000
m_test = X.shape[0] - m

#switch rows and columns and reshape
print(X.shape, y.shape)
X_train, X_test = X[:m].T, X[m:].T
y_train, y_test = y[:m].reshape(1,m), y[m:].reshape(1,m_test)
print(X_train.shape, y_train.shape, X_test.shape,  y_test.shape) 

(70000, 784) (70000,)
(784, 60000) (1, 60000) (784, 10000) (1, 10000)


In [1]:
def sigmoid(Z):
    """
    Sigmoid activation function.
    """
    return 1/(1+np.exp(-Z))

def der_sigmoid(Z):
    """
    Derivative of sigmoid activation function.
    """
    return sigmoid(Z) * (1 - sigmoid(Z))

def cross_entropy_loss(Y, Y_hat):
    """
    Binary cross entropy loss function.
    """
    n = Y.shape[1] 
    L = -(1./n) * (np.sum( np.multiply(np.log(Y_hat),Y) ) + np.sum( np.multiply(np.log(1-Y_hat),(1-Y)) ) )
    return L 


In [ ]:
class nn:
    
    def __init__(self, x, y, ):
        
        
        